In [3]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install tabula-py

   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ------ --------------------------------- 2.1/12.0 MB 19.5 MB/s eta 0:00:01
   ------------ --------------------------- 3.7/12.0 MB 16.8 MB/s eta 0:00:01
   ------------------- -------------------- 5.8/12.0 MB 10.4 MB/s eta 0:00:01
   ---------------------- ----------------- 6.8/12.0 MB 8.6 MB/s eta 0:00:01
   -------------------------- ------------- 7.9/12.0 MB 7.7 MB/s eta 0:00:01
   ----------------------------- ---------- 8.9/12.0 MB 7.3 MB/s eta 0:00:01
   --------------------------------- ------ 10.0/12.0 MB 6.9 MB/s eta 0:00:01
   ----------------------------------- ---- 10.7/12.0 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------  11.8/12.0 MB 6.5 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 6.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [31]:
pip install pandas beautifulsoup4 html5lib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from tkinter import *
import pyodbc as odbc
import matplotlib.pyplot as plt
from tkinter import Tk, Label
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from tkinter import ttk
from tkinter import messagebox
from tkcalendar import DateEntry
from datetime import datetime
import pandas as pd
import tabula
#import pdfplumber
import os
import html5lib
import bs4
import numpy as np
import re
import time
import csv
import tkinter as tk

# Connect to SQL Server
conn = odbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-7I9UC6G\\SQLEXPRESS;"
    "DATABASE=python;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()

def create_quotations_table():
    cursor.execute("""
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Quotations' AND xtype='U')
        CREATE TABLE Quotations (
            QuotationID INT PRIMARY KEY IDENTITY(1,1),
            CustomerName VARCHAR(255),
            ProductName VARCHAR(255),
            Quantity INT,
            Price DECIMAL(10,2),
            QuotationDate DATE DEFAULT GETDATE(),
            OrderStatus VARCHAR(20) DEFAULT 'Pending'
        );
    """)
    conn.commit()
    print("✅ Quotations table created.")

def insert_dummy_quotations():
    """Populate some sample quotations so the UI is not empty."""
    demo_data = [
        ("Akash Sharma", "USB Flash Drive 32GB", 10, 5.99, "Pending"),
        ("Maya Verma", "Wireless Mouse", 3, 12.50, "Confirmed"),
        ("Rahul Singh", "Laptop Stand", 4, 19.99, "Pending"),
        ("Kavita Joshi", "Bluetooth Headphones", 2, 29.99, "Pending"),
        ("Aarav Das", "Webcam HD", 1, 39.99, "Confirmed"),
    ]

    # Insert only if table is empty
    cursor.execute("SELECT COUNT(*) FROM Quotations")
    if cursor.fetchone()[0] == 0:
        for row in demo_data:
            cursor.execute(
                """
                INSERT INTO Quotations (CustomerName, ProductName, Quantity, Price, OrderStatus)
                VALUES (?,?,?,?,?)
                """,
                row,
            )
        conn.commit()
        print("✅ Dummy quotations inserted.")

# Ensure quotations table & sample data exist
create_quotations_table()
insert_dummy_quotations()

# 1. Create table if it doesn't exist
def create_inventory_table():
    cursor.execute("""
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Inventory' AND xtype='U')
        CREATE TABLE Inventory (
            ID INT PRIMARY KEY IDENTITY(1,1),
            ProductName VARCHAR(255),
            Quantity INT,
            Price DECIMAL(10,2),
            LastUpdated DATETIME DEFAULT GETDATE(),
            OrderConfirmed VARCHAR(20) DEFAULT 'Pending'
        );
    """)
    conn.commit()
    print("✅ Inventory table created successfully!")

# 2. Add unique constraint on ProductName
def add_unique_constraint():
    try:
        cursor.execute("""
            IF NOT EXISTS (
                SELECT * FROM sys.indexes 
                WHERE name='UQ_ProductName' AND object_id = OBJECT_ID('Inventory')
            )
            ALTER TABLE Inventory
            ADD CONSTRAINT UQ_ProductName UNIQUE (ProductName);
        """)
        conn.commit()
        print("✅ Unique constraint added to ProductName.")
    except Exception as e:
        print(f"❌ Error adding unique constraint: {e}")

# 3. Remove duplicates before adding constraint
def remove_duplicates():
    try:
        cursor.execute("""
            WITH CTE AS (
                SELECT *, ROW_NUMBER() OVER (PARTITION BY ProductName ORDER BY ID) AS RowNum
                FROM Inventory
            )
            DELETE FROM CTE WHERE RowNum > 1;
        """)
        conn.commit()
        print("✅ Duplicate products removed.")
    except Exception as e:
        print(f"❌ Error removing duplicates: {e}")

# 4. Temporarily disable FK constraints if blocking delete
def disable_foreign_keys():
    try:
        cursor.execute("""
            EXEC sp_msforeachtable "ALTER TABLE ? NOCHECK CONSTRAINT all"
        """)
        conn.commit()
        print("🔧 Foreign key constraints temporarily disabled.")
    except Exception as e:
        print(f"❌ Error disabling FK constraints: {e}")

def enable_foreign_keys():
    try:
        cursor.execute("""
            EXEC sp_msforeachtable "ALTER TABLE ? WITH CHECK CHECK CONSTRAINT all"
        """)
        conn.commit()
        print("🔧 Foreign key constraints re-enabled.")
    except Exception as e:
        print(f"❌ Error enabling FK constraints: {e}")

# 5. Reset inventory table
def reset_inventory():
    try:
        disable_foreign_keys()
        
        # Clean child table FIRST (to prevent FK conflict)
        cursor.execute("DELETE FROM Sales;")
        
        # Now clean parent table
        cursor.execute("DELETE FROM Inventory;")
        cursor.execute("DBCC CHECKIDENT ('Inventory', RESEED, 0);")
        
        conn.commit()
        enable_foreign_keys()
        print("✅ Inventory table reset successfully.")
    except Exception as e:
        print(f"❌ Error resetting inventory: {e}")
        
# 6. Insert dummy data
def insert_dummy_data():
    products = [
        ('USB Flash Drive 32GB', 50, 5.99, 'Pending'),
        ('Wireless Mouse', 25, 12.50, 'Confirmed'),
        ('Laptop Stand', 40, 19.99, 'Pending'),
        ('Bluetooth Headphones', 8, 29.99, 'Pending'),
        ('Webcam HD', 4, 39.99, 'Confirmed'),
        ('Mechanical Keyboard', 9, 59.99, 'Cancelled'),
        ('Smartphone Tripod', 0, 14.99, 'Pending'),  # ✅ Fixed: removed extra argument
        ('Portable Hard Drive 1TB', 0, 79.99, 'Pending'),
        ('Pack of AA Batteries', 60, 7.49, 'Confirmed'),
        ('HDMI Cable 2m', 12, 4.99, 'Pending')
    ]

    try:
        for product in products:
            cursor.execute("""
                INSERT INTO Inventory (ProductName, Quantity, Price, OrderConfirmed)
                VALUES (?, ?, ?, ?)
            """, product)
        conn.commit()
        print("✅ Dummy data inserted successfully!")
    except Exception as e:
        print(f"❌ Error inserting dummy data: {e}")

# 7. Ensure 'OrderConfirmed' column exists (redundant in your case, but kept for safety)
def add_order_confirmed_column():
    try:
        cursor.execute("""
            IF NOT EXISTS (
                SELECT * FROM INFORMATION_SCHEMA.COLUMNS 
                WHERE TABLE_NAME = 'Inventory' AND COLUMN_NAME = 'OrderConfirmed'
            )
            ALTER TABLE Inventory ADD OrderConfirmed VARCHAR(20) DEFAULT 'Pending';
        """)
        conn.commit()
        print("✅ OrderConfirmed column ensured.")
    except Exception as e:
        print(f"❌ Error ensuring OrderConfirmed column: {e}")

# 8. Insert single item manually
def insert_inventory(product_name, quantity, price, order_status="Pending"):
    try:
        cursor.execute(
            "INSERT INTO Inventory (ProductName, Quantity, Price, OrderConfirmed) VALUES (?, ?, ?, ?)",
            (product_name, quantity, price, order_status)
        )
        conn.commit()
        print("✅ Product added successfully!")
    except Exception as e:
        print(f"❌ Error adding product: {e}")

# 9. Update status
def update_order_status(product_name, status):
    try:
        cursor.execute(
            "UPDATE Inventory SET OrderConfirmed = ? WHERE ProductName = ?",
            (status, product_name)
        )
        conn.commit()
        print(f"✅ Order status for '{product_name}' updated to: {status}")
    except Exception as e:
        print(f"❌ Error updating order status: {e}")

def rename_orderconfirmed_to_status():
    try:
        cursor.execute("""
            IF EXISTS (
                SELECT * FROM INFORMATION_SCHEMA.COLUMNS
                WHERE TABLE_NAME = 'Inventory' AND COLUMN_NAME = 'OrderConfirmed'
            )
            EXEC sp_rename 'Inventory.OrderConfirmed', 'OrderStatus', 'COLUMN';
        """)
        conn.commit()
        print("✅ Column 'OrderConfirmed' successfully renamed to 'OrderStatus'.")
    except Exception as e:
        print(f"❌ Error renaming column: {e}")

# 10. Setup function
def setup_inventory():
    create_inventory_table()
    #rename_orderconfirmed_to_status()
    add_order_confirmed_column()
    remove_duplicates()
    add_unique_constraint()
    reset_inventory()
    insert_dummy_data()

# Run setup
setup_inventory()



def create_sales_table():
    cursor.execute("""
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Sales' AND xtype='U')
        CREATE TABLE Sales (
            SaleID INT PRIMARY KEY IDENTITY(1,1),
            ProductID INT FOREIGN KEY REFERENCES Inventory(ID),
            QuantitySold INT,
            TotalPrice DECIMAL(10,2),
            SaleDate DATETIME DEFAULT GETDATE()
        );
    """)
    conn.commit()
    print("✅ Sales table created successfully!")

#create_sales_table()

def insert_sales(product_id, quantity_sold):
    #Get price from Inventory
    cursor.execute("SELECT Price FROM Inventory WHERE ID = ?", (product_id,))
    price = cursor.fetchone()
    
    if price:
        total_price = price[0] * quantity_sold
        
        ## Insert sales record
        cursor.execute("INSERT INTO Sales (ProductID, QuantitySold, TotalPrice) VALUES (?, ?, ?)",
                       (product_id, quantity_sold, total_price))
        
        ## Update inventory stock
        cursor.execute("UPDATE Inventory SET Quantity = Quantity - ? WHERE ID = ?", (quantity_sold, product_id))
        
        conn.commit()
        print("✅ Sale recorded successfully!")
    else:
         print("⚠️ Product ID not found!")

def create_employees_table():
    cursor.execute("""
    IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Employees' AND xtype='U')
    CREATE TABLE Employees (
    ID INT PRIMARY KEY IDENTITY(1,1),
    Name VARCHAR(255),
    Position VARCHAR(255),
    Salary DECIMAL(10,2),
    HireDate DATE DEFAULT GETDATE()
    );
    """)
    conn.commit()
    print("✅ Employees table created successfully!")

# Function to insert sample employees
def insert_sample_employees():
    sample_data = [
        ("Amit Patel", "Manager", 75000, "2022-05-01"),
        ("Sofia Sharma", "Sales Executive", 45000, "2023-02-15"),
        ("Ravi Kumar", "Technician", 50000, "2021-11-10"),
        ("Priya Mehta", "Accountant", 60000, "2020-09-25"),
        ("Vikram Singh", "HR Specialist", 55000, "2023-07-03")
    ]
    
    cursor.executemany("INSERT INTO Employees (Name, Position, Salary, HireDate) VALUES (?, ?, ?, ?)", sample_data)
    conn.commit()
    print("✅ Sample employee records inserted successfully!")

## Call the functions
#create_employees_table()
#insert_sample_employees()

# Close connection
#cursor.close()
#conn.close()

def create_supplier_table():
    cursor.execute("""
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Suppliers' AND xtype='U')
        CREATE TABLE Suppliers (
            ID INT PRIMARY KEY IDENTITY(1,1),
            Name VARCHAR(255),
            Company VARCHAR(255),
            Contact VARCHAR(15),
            Materials VARCHAR(500)
        );
    """)
    conn.commit()
    print("✅ Suppliers table created successfully!")

def insert_sample_suppliers():
    sample_suppliers = [
        ("Rajesh Patel", "TechCorp Pvt Ltd", "9876543210", "Electronic Components, Sensors"),
        ("Neha Sharma", "Green Energy Ltd", "9123456789", "Solar Panels, Batteries"),
        ("Amit Gupta", "SteelWorks Industries", "8997766554", "Steel Frames, Pipes"),
        ("Sofia Mehta", "Alpha Traders", "9911223344", "Wires, Cables, Electrical Tools"),
        ("Ravi Kumar", "SuperTech Solutions", "9001122334", "Computers, Hardware Parts")
    ]

    for supplier in sample_suppliers:
        cursor.execute("SELECT COUNT(*) FROM Suppliers WHERE Name = ? AND Company = ?", (supplier[0], supplier[1]))
        existing_count = cursor.fetchone()[0]

        if existing_count == 0:  # If supplier does not exist, insert the record
            cursor.execute("INSERT INTO Suppliers (Name, Company, Contact, Materials) VALUES (?, ?, ?, ?)", supplier)
            
    conn.commit()
    print("✅ Sample supplier records inserted successfully (No duplicates added)!")

create_supplier_table()
insert_sample_suppliers()

def create_orders_table():
    cursor.execute("""
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Orders' AND xtype='U')
        CREATE TABLE Orders (
            OrderID INT PRIMARY KEY IDENTITY(1,1),
            CustomerID INT,
            ProductID INT,
            Quantity INT,
            TotalAmount DECIMAL(10,2),
            OrderDate DATETIME DEFAULT GETDATE(),
            Status VARCHAR(50) DEFAULT 'Pending',
            PaymentMethod VARCHAR(50),
            ShippingAddress TEXT,
            LastUpdated DATETIME DEFAULT GETDATE()
        );
    """)
    conn.commit()
    print("✅ Orders table created successfully!")

create_orders_table()

# Function to make frames clickable
def make_frame_clickable(frame, title, callback):
    frame.bind("<Button-1>", lambda e: callback(title))
    for child in frame.winfo_children():
        child.bind("<Button-1>", lambda e: callback(title))

root=Tk()
root.title('Dashboard')
root.geometry('1350x690+0+0')  # +0+0 means distance of frame will be 0 from x and y axis
root.config(bg='#5761B2')

subtitleLabel = tk.Label(root, font=('Arial', 14))
subtitleLabel.pack()

def update_datetime():
    current_time = time.strftime("%I:%M:%S %p")  # 12-hour format time with AM/PM
    current_date = time.strftime("%d-%m-%Y")
    subtitleLabel.config(text=f"Welcome Admin\t\t Date: {current_date}\t\t Time: {current_time}")
    root.after(1000, update_datetime)  # call this function again after 1000ms (1 second)

update_datetime()  # initial call to start the clock


from tkinter import Toplevel

notifications = []

notif_frame = ttk.Frame(root)
notif_frame.pack(pady=20)

# Function to fetch recent changes from database
def fetch_notifications():
    cursor.execute("SELECT TOP 5 'New sale recorded!' FROM Sales ORDER BY SaleDate DESC")
    new_sales = cursor.fetchall()
    
    cursor.execute("SELECT TOP 5 'Inventory updated!' FROM Inventory ORDER BY LastUpdated DESC")
    new_inventory = cursor.fetchall()
    
    cursor.execute("SELECT TOP 5 'Supplier added successfully!' FROM Suppliers ORDER BY ID DESC")
    new_suppliers = cursor.fetchall()

    latest_notifications = [notif[0] for notif in (new_sales + new_inventory + new_suppliers)]
    
    return latest_notifications

def update_notification_display(notif_window, scroll_frame):
    for widget in scroll_frame.winfo_children():
        widget.destroy()  # Clear old notifications

    Label(scroll_frame, text="Recent Notifications:", font=('times new roman', 15, 'bold')).pack(pady=10)

    notifications = fetch_notifications()
    
    for notif in notifications:
        Label(scroll_frame, text=f"- {notif}", font=('times new roman', 13), fg="white", bg="#2C3E50", padx=10, pady=5).pack(anchor="w", padx=20)

    # Re-run this function every 5 seconds
    notif_window.after(5000, lambda: update_notification_display(notif_window, scroll_frame))


# Function to open notifications window
def open_notifications():
    notif_window = Toplevel(root)
    notif_window.title("Notifications")
    notif_window.geometry("400x300+500+200")
    notif_window.config(bg="#2C3E50") 
    
    Label(notif_window, text="🔔 Notifications:", font=('times new roman', 20, 'bold')).pack(pady=10)

    canvas = Canvas(notif_window, bg="#2C3E50")
    scrollbar = Scrollbar(notif_window, orient="vertical", command=canvas.yview)
    scroll_frame = Frame(canvas, bg="#2C3E50")

    scroll_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))

    canvas.create_window((0, 0), window=scroll_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)

    update_notification_display(notif_window, scroll_frame)
   
    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    Button(notif_window, text="Close", command=notif_window.destroy).pack(pady=15)

                                                                                                                                  
bg_image=PhotoImage(file='inventory (2).png')
titleLabel=Label(root, image=bg_image ,compound=LEFT,  text='Inventory Managment System', font=('times new roman', 40, 'bold'), bg='#010c48', fg='white', anchor='w', padx=20)
titleLabel.place(x=0,y=0, relwidth=1)

bell_icon = PhotoImage(file="bell.png")  # Ensure you have a bell.png file
bell_button = Button(root, image=bell_icon, command=open_notifications, bd=2)
bell_button.place(x=1120, y=17)  # Adjust position as needed970, y=150

LogoutButton=Button(root, text='Logout', font=('times new roman', 20, 'bold'))
LogoutButton.place(x=1200,y=9)

subtitle=Label(root, text='Welcome Admin\t\t Date: 08-07-2024\t\t Time: 12:36:13 pm', font=('times new roman', 15),bg='#4d636d', fg='white')
subtitle.place(x=0, y=68, relwidth=1)

leftFrame=Frame(root)
leftFrame.place(x=0, y=96, width=200, height=595)

logoImage=PhotoImage(file='checklist (1).png')
ImageLabel=Label(leftFrame, image=logoImage)
ImageLabel.pack()

menuLabel=Label(leftFrame, text='Menu', font=('times new roman',20),bg='#009688')
menuLabel.pack(fill=X)

emp_image=PhotoImage(file='employee.png')
employee_button=Button(leftFrame, image=emp_image ,compound=LEFT,  text='Employees', font=('times new roman', 20, 'bold'))
employee_button.pack(fill=X)

product_image=PhotoImage(file='box.png')
Product_button=Button(leftFrame, image=product_image, compound=LEFT, text='Invoice', font=('times new roman', 20, 'bold'), command=lambda: product_form(root))
Product_button.pack(fill=X)

panel_image=PhotoImage(file='solar.png')
Panel_button=Button(leftFrame, image=panel_image ,compound=LEFT,  text='Panels', font=('times new roman', 20, 'bold'))
Panel_button.pack(fill=X)

Inventory_image=PhotoImage(file='inventory (1).png')
Inventory_button=Button(leftFrame, image=Inventory_image ,compound=LEFT, text='Inventory', font=('times new roman', 20, 'bold'))
Inventory_button.pack(fill=X)

billing_image=PhotoImage(file='bill.png')
Billing_button=Button(leftFrame, image=billing_image ,compound=LEFT, text='Billing', font=('times new roman', 20, 'bold'))
Billing_button.pack(fill=X)

sales_image=PhotoImage(file='increase.png')
sales_button=Button(leftFrame, image=sales_image ,compound=LEFT, text='Sales', font=('times new roman', 20, 'bold'))
sales_button.pack(fill=X)

help_image=PhotoImage(file='chat.png')
help_button=Button(leftFrame, image=help_image ,compound=LEFT, text='Order Status', font=('times new roman', 20, 'bold'), command=lambda: order_status1(root))
help_button.pack(fill=X)

exit_image=PhotoImage(file='exit.png')
exit_button=Button(leftFrame, image=exit_image ,compound=LEFT, text='Exit', font=('times new roman', 20, 'bold'))
exit_button.pack(fill=X)

def product_form(root):
    global back_image, df, file_name, file_path

    product_window = tk.Toplevel()
    product_window.title("Invoice")
    product_window.attributes("-fullscreen", True)
    product_window.grab_set()

    product_frame = Frame(product_window, bg='white')
    product_frame.place(relx=0, rely=0, relwidth=1, relheight=1)

    top_close_button = Button(product_window, text='X', font=('Arial', 14, 'bold'),
                              bg='red', fg='white', cursor='hand2', bd=0,
                              command=product_window.destroy)
    top_close_button.place(relx=1.0, x=-15, y=20, anchor='ne', width=40, height=40)

    heading_label = Label(product_frame, text='Get Inventory Files',
                          font=('times new roman', 18, 'bold'),
                          bg='#0f4d7d', fg='white', anchor='w')
    heading_label.place(x=20, y=20, relwidth=0.95, height=40)

    search_frame = LabelFrame(product_frame, text='Search Product',
                              font=('times new roman', 14), padx=10, pady=10)
    search_frame.place(relx=0.05, rely=0.12, relwidth=0.9, height=80)

    for i in range(4):
        search_frame.columnconfigure(i, weight=1)

    search_combobox = ttk.Combobox(search_frame,
                                   values=('HTML', 'Excel', 'CSV', 'TXT', 'PDF'),
                                   state='readonly', font=('times new roman', 14))
    search_combobox.grid(row=0, column=0, padx=5, pady=10, sticky='ew')
    search_combobox.set("Select File Type")

    output_frame = Frame(product_frame, bg='white')
    output_frame.place(relx=0.05, rely=0.25, relwidth=0.9, relheight=0.68)

    output_text = Text(output_frame, bg='white', font=('Consolas', 12), wrap='none')
    output_text.pack(side=LEFT, fill=BOTH, expand=True)
    heading_label=Label(output_text,text='Result Of Searched Files',font=('times new roman',16,'bold'),bg='#0f4d7d',fg='white')
    heading_label.place(x=0,y=0,relwidth=1)

    scroll_y = Scrollbar(output_frame, orient=VERTICAL, command=output_text.yview)
    scroll_y.pack(side=RIGHT, fill=Y)

    scroll_x = Scrollbar(output_frame, orient=HORIZONTAL, command=output_text.xview)
    scroll_x.pack(side=BOTTOM, fill=X)

    output_text.config(yscrollcommand=scroll_y.set, xscrollcommand=scroll_x.set)

 #   output_text.insert('1.0', 'Result Of Searched Files' + ' ' * 220 + '\n')

  #  output_text.tag_add('heading', '1.0', '1.end')
   # output_text.tag_config('heading', font=('times new roman', 16, 'bold'),
    #                       background='#0f4d7d', foreground='white')
  #  output_text.tag_configure("center", justify='center')
   # output_text.tag_add("center", "1.0", "1.end")

    file_types = {
        "HTML": (".html", pd.read_html),
        "Excel": (".xlsx", pd.read_excel),
        "CSV": (".csv", pd.read_csv),
        "TXT": (".txt", pd.read_csv),
        "PDF": (".pdf", None),
    }

    def pdf_to_excel(file_path, output_excel_path):
        with pdfplumber.open(file_path) as pdf:
            all_tables = []
            for page in pdf.pages:
                tables = page.extract_tables()
                for table in tables:
                    if table and len(table) > 1:
                        df = pd.DataFrame(table[1:], columns=table[0])
                        all_tables.append(df)
                try:
                    df = pd.DataFrame(table[1:], columns=table[0])
                except Exception:
                    # Fallback if header row is malformed
                    df = pd.DataFrame(table)
                all_tables.append(df)

    
    def analyze_pdf_bill(file_path):
        global text
        with pdfplumber.open(file_path) as pdf:
            all_text = ""
            tables = []
            
            for page_num, page in enumerate(pdf.pages):
                text = page.extract_text()
                if text:
                    print(f"\n--- Page {page_num + 1} Text ---")
                    print(text)
                    all_text += text + "\n"
                
                # Extract any tables found on the page
                page_tables = page.extract_tables()
                for table in page_tables:
                    df = pd.DataFrame(table[1:], columns=table[0])  # Use first row as header
                    print(f"\n--- Page {page_num + 1} Table ---")
                    print(df)
                    tables.append(df)
                    
            if not tables:
                print(f"No tables found on page {page_num + 1}")
        
            return all_text, tables

    # Example usage
  #  pdf_file = "path/to/your/bill.pdf"
   # text, tables = analyze_pdf_bill(pdf_file)


    def extract_metadata(text):
        invoice_no = re.search(r"Invoice\s*No[:\-]?\s*(\w+)", text)
        total_amt = re.search(r"Total\s*Amount[:\-]?\s*(\d+\.\d{2})", text)
        return {
            "invoice_no": invoice_no.group(1) if invoice_no else None,
            "total_amt": total_amt.group(1) if total_amt else None,
        }
    
    # Example usage:
#    metadata = extract_metadata(text)
#    print("Invoice No:", metadata['invoice_no'])
 #   print("Total Amount:", metadata['total_amt'])

    def clean_dataframe(df):
        try:
            df.dropna(how='all', inplace=True)
            df.fillna(value=0, inplace=True)
            for col in df.select_dtypes(include=['object']):
                df[col] = df[col].str.strip()
            for col in df.columns:
                try:
                    df[col] = pd.to_numeric(df[col])
                except:
                    continue
            df.drop_duplicates(inplace=True)
            return df
        except Exception as e:
            print(f"An error occurred during cleaning: {e}")
            return None
           

    def load_file(event=None):
        global df, file_name, file_path
        file_type = search_combobox.get()
        if file_type not in file_types:
            messagebox.showerror("Error", "Unsupported file type selected.")
            return

        ext, reader = file_types[file_type]
        file_path = filedialog.askopenfilename(title="Select a file", filetypes=[(f"{file_type} files", f"*{ext}")])
        if not file_path:
            messagebox.showwarning("No file", "No file was selected.")
            return

        file_name = os.path.basename(file_path)
        entry_var.set(file_name)

        try:
            if file_type == "HTML":
                df_list = reader(file_path)
                if not df_list:
                    raise ValueError("No tables found in HTML.")
                df = df_list[0]
            elif file_type == "PDF":
                output_excel = "output_file.xlsx"
                pdf_to_excel(file_path, output_excel)
                df = pd.read_excel(output_excel)
            else:
                df = reader(file_path)

            df = clean_dataframe(df)

            output_text.delete("1.0", tk.END)
            output_text.insert(tk.END, f"Loaded {file_name} ({len(df)} rows):\n\n")
            output_text.insert(tk.END, df.head().to_string(index=False))

        except Exception as e:
            messagebox.showerror("Read Error", f"Could not read file:\n{e}")
            
    def add_file_to_sql():
        if df is None or df.empty:
            messagebox.showwarning("No Data", "Please load a valid file with data first.")
            return
    
        try:
            conn = pyodbc.connect(
                "DRIVER={ODBC Driver 17 for SQL Server};"
                "SERVER=LAPTOP-AQMG9SG5\\SQLEXPRESS;"
                "DATABASE=files;"
                "Trusted_Connection=yes;"
            )
            cursor = conn.cursor()
    
            # Create table if not exists
            cursor.execute("""
                IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='NEWTAB' AND xtype='U')
                CREATE TABLE NEWTAB(
                   cat_num INT PRIMARY KEY IDENTITY(1,1),
                   make VARCHAR(50),
                   material_name VARCHAR(50),
                   quantity DECIMAL(10,2),
                   discount DECIMAL(10,2),
                   rate_per_pannel DECIMAL(10,2),
                   total DECIMAL(10,2)
                );
            """)
            conn.commit()
    
            inserted = 0
            updated = 0
    
            for _, row in df.iterrows():
                try:
                    make = str(row.get('make', '')).strip()
                    name = str(row.get('material_name', '')).strip()
                    qty = Decimal(row.get('quantity', 0))
                    discount = Decimal(row.get('discount', 0))
                    rate = Decimal(row.get('rate_per_pannel', 0))
    
                    # Check if a matching row already exists
                    cursor.execute("""
                        SELECT quantity, discount FROM NEWTAB
                        WHERE make = ? AND material_name = ? AND rate_per_pannel = ?
                    """, (make, name, rate))
                    existing = cursor.fetchone()
    
                    if existing:
                        # Row exists, update by adding new quantity and discount
                        existing_qty, existing_discount = existing
                        new_qty = existing_qty + qty
                        new_discount = existing_discount + discount
                        new_total = round(new_qty * rate, 2)
    
                        cursor.execute("""
                            UPDATE NEWTAB
                            SET quantity = ?, discount = ?, total = ?
                            WHERE make = ? AND material_name = ? AND rate_per_pannel = ?
                        """, (new_qty, new_discount, new_total, make, name, rate))
                        updated += 1
                    else:
                        # Insert new row
                        total = round(qty * rate, 2)
                        cursor.execute("""
                            INSERT INTO NEWTAB (make, material_name, quantity, discount, rate_per_pannel, total)
                            VALUES (?, ?, ?, ?, ?, ?)
                        """, (make, name, qty, discount, rate, total))
                        inserted += 1
    
                except Exception as row_e:
                    print(f"Row processing failed: {row_e}")
    
            conn.commit()
            messagebox.showinfo("Success", f"{inserted} row(s) inserted, {updated} row(s) updated in SQL.")
    
        except Exception as e:
            messagebox.showerror("Database Error", f"Failed to insert/update data:\n{e}")
    
        finally:
            if 'conn' in locals():
                conn.close()

    search_combobox.bind("<<ComboboxSelected>>", load_file)
    entry_var = tk.StringVar()
    file_id_var = StringVar()
    search_entry = Entry(search_frame, font=('times new roman', 14, 'bold'),
                         bg='lightyellow', textvariable=entry_var)
    search_entry.grid(row=0, column=1, padx=5, sticky='ew')

    search_button = Button(search_frame, text='Add', font=('times new roman', 14),
                           cursor='hand2', fg='white', bg='#0f4d7d', command=lambda:  add_file_to_sql())
    search_button.grid(row=0, column=2, padx=5, sticky='ew')



graphtitle=Label(root, text='📊 Buisness Overview', font=('times new roman', 17, 'bold'), bg='#7BD5F5')
graphtitle.place(x=970, y=130)

fig, ax = plt.subplots(figsize=(5.5, 5.1))
canvas = FigureCanvasTkAgg(fig, master=root)
canvas.get_tk_widget().place(x=780, y=170)

def update_graph(data, title, chart_type="bar"):
    ax.clear()
    ax.set_title(title, fontsize=14)

    if not data:
        ax.text(0.5, 0.5, "No data to display", fontsize=12, ha='center', va='center')
        canvas.draw()
        canvas.get_tk_widget().update_idletasks()
        return

    labels = [str(row[0]) for row in data]

    def safe_float(val):
        try:
            return float(val)
        except (ValueError, TypeError):
            return 0
            
    values = [safe_float(row[1]) for row in data]

    # Plot based on chart type
    if chart_type == "bar":
        ax.bar(labels, values, color="#61AFEF")
    elif chart_type == "line":
        ax.plot(labels, values, color="#EF476F", marker='o', linestyle='-')
    elif chart_type == "pie":
        if sum(values) == 0:
            ax.text(0.5, 0.5, "No valid data for pie chart", fontsize=12, ha='center', va='center')
        else:
            ax.pie(values, labels=labels, autopct='%1.1f%%', startangle=140)
            ax.axis('equal')  # Make pie chart circular
    else:
        ax.text(0.5, 0.5, "Invalid chart type", fontsize=12, ha='center', va='center')

    # Styling for bar and line charts
    if chart_type in ("bar", "line"):
        # Rotate x-axis labels
        plt.setp(ax.get_xticklabels(), rotation=45, ha='right', va='top')
        # Padding
        ax.tick_params(axis='x', pad=10)

    # Adjust layout to prevent clipping
    fig.tight_layout()

    # Redraw the canvas
    canvas.draw()
    canvas.get_tk_widget().update_idletasks()


def show_data_in_window(title, data, columns):
    if not data or not columns:
        print("No data or columns provided.")
        return

    def search_tree():
        query = search_var.get().lower()
        for row in tree.get_children():
            tree.delete(row)

        for item in data:
            if any(query in str(value).lower() for value in item):
                tree.insert("", "end", values=[str(value) for value in item])

    data_window = Toplevel(root)
    data_window.title(title)
    data_window.geometry(f"{root.winfo_screenwidth()}x{root.winfo_screenheight()}")
    data_window.state('zoomed')
    data_window.config(bg="#96C3EB")

    Label(data_window, text=title, font=('times new roman', 30, 'bold'), bg="#158FAD", fg="white", pady=20).pack()

    search_frame = Frame(data_window, bg="#96C3EB")
    search_frame.pack(pady=5)

    search_var = StringVar()
    Entry(search_frame, textvariable=search_var, font=('times new roman', 14), width=40).pack(side=LEFT, padx=10)
    Button(search_frame, text="Search", font=('times new roman', 12, 'bold'),
           command=search_tree).pack(side=LEFT)

    frame = Frame(data_window, bg="#96C3EB", bd=5, relief=SOLID)
    frame.pack(expand=True, fill="both", padx=20, pady=20)

    v_scrollbar = Scrollbar(frame, orient=VERTICAL)
    h_scrollbar = Scrollbar(frame, orient=HORIZONTAL)
    v_scrollbar.pack(side=RIGHT, fill=Y)
    h_scrollbar.pack(side=BOTTOM, fill=X)
    
    tree = ttk.Treeview(frame, columns=columns, show="headings", yscrollcommand=v_scrollbar.set , xscrollcommand=h_scrollbar.set)
    tree.pack(expand=True, fill="both")
    v_scrollbar.config(command=tree.yview)
    h_scrollbar.config(command=tree.xview)

    for col in columns:
        tree.heading(col, text=col, anchor=CENTER)
        tree.column(col, anchor=CENTER, width=200)

    style = ttk.Style()
    style.configure("Treeview", font=("times new roman", 15), rowheight=30, background="#ECF0F1", foreground="black")
    style.configure("Treeview.Heading", font=("times new roman", 18, "bold"), background="black", foreground="black")

    for item in data:
        tree.insert("", "end", values=[str(value) for value in item])

    Button(data_window, text="Close", font=('times new roman', 15, 'bold'),
           command=data_window.destroy).pack(pady=20)

def fetch_employee_data():
    cursor.execute("SELECT ID, Name, Position, Salary, HireDate FROM Employees")
    employees = cursor.fetchall()
    return employees

def open_employee_window(title):
    employees = fetch_employee_data() 
    show_data_in_window(title, employees, ["ID","Name","Position","Salary","Hire Date"])
    update_graph([(row[1], row[3]) for row in employees], "Employee Salaries", chart_type="bar")

emp_frame=Frame(root, bg='#2C3E50', bd=3,  relief=RIDGE)
emp_frame.place(x=230, y=125, height=170, width=230)
total_emp_image=PhotoImage(file='team.png')
total_emp_image_label=Label(emp_frame, image=total_emp_image, bg='#2C3E50')
total_emp_image_label.pack()
total_emp_label=Label(emp_frame, text='Staffing Overview', font=('times new roman', 15, 'bold') ,bg='#2C3E50', fg='white')
total_emp_label.pack()
total_emp_count_label=Label(emp_frame, text='0', bg='#2C3E50',fg='white', font=('times new roman', 20, 'bold'))
total_emp_count_label.pack()
make_frame_clickable(emp_frame, 'Staffing Overview', open_employee_window)

def fetch_supplier_data():
    cursor.execute("SELECT ID, Name, Company, Contact, Materials FROM Suppliers")
    suppliers = cursor.fetchall()
    return suppliers
    
def open_supplier_window(title):
    from collections import Counter 
    suppliers = fetch_supplier_data()  # Fetch supplier details
    show_data_in_window(title, suppliers, ["ID","Name","Company","Contact","Materials"])
    company_counts = Counter(row[2] for row in suppliers)
    update_graph(list(company_counts.items()), "Suppliers by Company", chart_type="pie")

sup_frame=Frame(root, bg='#2C3E50', bd=3, relief=RIDGE)
sup_frame.place(x=520, y=125, height=170, width=230)
total_sup_image=PhotoImage(file='supplier.png')
total_sup_image_label=Label(sup_frame, image=total_sup_image, bg='#2C3E50')
total_sup_image_label.pack()
total_sup_label=Label(sup_frame, text='Supplier Network', font=('times new roman', 15, 'bold') ,bg='#2C3E50', fg='white')
total_sup_label.pack()
total_sup_count_label=Label(sup_frame, text='0', bg='#2C3E50',fg='white', font=('times new roman', 20, 'bold'))
total_sup_count_label.pack()
make_frame_clickable(sup_frame, 'Vendor Ecosystem', open_supplier_window)

def open_revenue_window(title):
    revenue_data = [("Q1", 20000), ("Q2", 30000), ("Q3", 40000), ("Q4", 50000)]
    update_graph(revenue_data, "Quarterly Revenue", chart_type="line")
    messagebox.showinfo("Coming Soon", "Revenue insights coming soon!")

revenue_frame=Frame(root, bg='#2C3E50', bd=3, relief=RIDGE)
revenue_frame.place(x=230, y=310, height=170, width=230)
total_revenue_image=PhotoImage(file='revenue.png')
total_revenue_image_label=Label(revenue_frame, image=total_revenue_image, bg='#2C3E50')
total_revenue_image_label.pack()
total_revenue_label=Label(revenue_frame, text='Revenue Insights', font=('times new roman', 15, 'bold') ,bg='#2C3E50', fg='white')
total_revenue_label.pack()
total_revenue_count_label=Label(revenue_frame, text='0', bg='#2C3E50',fg='white', font=('times new roman', 20, 'bold'))
total_revenue_count_label.pack()
make_frame_clickable(revenue_frame, 'Revenue Insights', open_revenue_window)


def show_data_in_product_window(title, data, columns):
    data_window = Toplevel(root)
    data_window.title(title)
    data_window.geometry(f"{root.winfo_screenwidth()}x{root.winfo_screenheight()}")
    data_window.state('zoomed')
    data_window.config(bg="#96C3EB")

    Label(data_window, text=title, font=('times new roman', 30, 'bold'), bg="#158FAD", fg="white", pady=20).pack(fill=X)

     # === Search and Date Filter Bar ===
    filter_frame = Frame(data_window, bg="#96C3EB")
    filter_frame.pack(pady=10, padx=20, fill=X)

    Label(filter_frame, text="Search Customer", font=('times new roman', 14), bg="#96C3EB").pack(side=LEFT)
    search_var = StringVar()
    search_entry = Entry(filter_frame, textvariable=search_var, font=('times new roman', 14), width=30)
    search_entry.pack(side=LEFT, padx=10)

    # Date range selection
    Label(filter_frame, text="From:", font=('times new roman', 14), bg="#96C3EB").pack(side=LEFT, padx=(20, 5))
    from_date = DateEntry(filter_frame, font=('times new roman', 14), date_pattern='yyyy-mm-dd')
    from_date.pack(side=LEFT)

    Label(filter_frame, text="To:", font=('times new roman', 14), bg="#96C3EB").pack(side=LEFT, padx=(20, 5))
    to_date = DateEntry(filter_frame, font=('times new roman', 14), date_pattern='yyyy-mm-dd')
    to_date.pack(side=LEFT)

    Button(filter_frame, text="Filter", font=('times new roman', 12), command=lambda: filter_table(), bg="#158FAD", fg="white"
          ).pack(side=LEFT, padx=20)

 
    # === Scrollable Canvas Setup ===
    canvas_frame = Frame(data_window)
    canvas_frame.pack(expand=True, fill=BOTH)

    canvas = Canvas(canvas_frame, bg="#96C3EB")
    canvas.pack(side=LEFT, fill=BOTH, expand=True)

    v_scroll = Scrollbar(canvas_frame, orient=VERTICAL, command=canvas.yview)
    v_scroll.pack(side=RIGHT, fill=Y)

    h_scroll = Scrollbar(data_window, orient=HORIZONTAL, command=canvas.xview)
    h_scroll.pack(side=BOTTOM, fill=X)
   
    canvas.configure(yscrollcommand=v_scroll.set, xscrollcommand=h_scroll.set)

    table_frame = Frame(canvas, bg="#EAF4FF")
    table_window= canvas.create_window((0, 0), window=table_frame, anchor="nw")

    def update_scrollregion(event=None):
        canvas.configure(scrollregion=canvas.bbox("all"))
        # Optional: Resize canvas window width
        canvas.itemconfig(table_window, width=canvas.winfo_width())

    table_frame.bind("<Configure>", update_scrollregion)
    canvas.bind("<Configure>", update_scrollregion)


    product_vars = {}
    product_names = {}

    # === Function to Populate the Table ===
    def render_table(filtered_data):
        for widget in table_frame.winfo_children():
            widget.destroy()

        headers = ["Select"] + columns
        for col_index, header in enumerate(headers):
            col_width = 6 if col_index == 0 else 18  # ✅ Narrow checkbox column
            Label(table_frame, text=header, font=('times new roman', 14, 'bold'), bg="#158FAD", fg="white", width=col_width, anchor="center"
                 ).grid(row=0, column=col_index, sticky="nsew", padx=2, pady=4)

        for row_index, row_data in enumerate(filtered_data):
            index = row_index
            var = IntVar()
            product_vars[index] = var
            product_names[index] = row_data[0]

            Checkbutton(table_frame, variable=var, bg="#EAF4FF").grid(row=row_index + 1, column=0, sticky="w", padx=5, pady=2)

            for col_index, cell_value in enumerate(row_data):
                Label(table_frame, text=str(cell_value), font=('times new roman', 12), bg="#EAF4FF",
                      anchor="center", justify=CENTER, width=15
                     ).grid(row=row_index + 1, column=col_index + 1, sticky="nsew", padx=2, pady=2)

    # === Filter Handler ===
    def filter_table(*args):
        query = search_var.get().strip().lower()
        from_d = from_date.get_date()
        to_d = to_date.get_date()

        def match(row):
            name_match = query in row[0].lower()
            try:
                row_date = datetime.strptime(row[3], '%Y-%m-%d').date()  # Assuming LastUpdated is at index 3
                date_match = from_d <= row_date <= to_d
            except Exception:
                date_match = False
            return name_match and date_match

        filtered = [row for row in data if match(row)]
        render_table(filtered)

    search_var.trace_add("write", filter_table)
    render_table(data)

    def _on_mousewheel(event):
        canvas.yview_scroll(int(-1*(event.delta/120)), "units")
        canvas.xview_scroll(int(-1*(event.delta/120)), "units") 
    canvas.bind("<MouseWheel>", _on_mousewheel)

    # === Process Button Action ===
    def update_selected_rows(new_status):
        selected_rows = [data[i] for i, v in product_vars.items() if v.get() == 1]
        if not selected_rows:
            messagebox.showwarning("No Selection", "Please select at least one quotation.")
            return

        try:
            for row in selected_rows:
                cursor.execute(
                    "UPDATE Quotations SET OrderStatus = ? WHERE CustomerName = ? AND ProductName = ? AND QuotationDate = ?",
                    (new_status, row[0], row[1], row[4]),
                )
            conn.commit()
            messagebox.showinfo("Success", f"Selected quotations marked as '{new_status}'.")
            # refresh data
            refreshed = fetch_customer_quotations()
            render_table(refreshed)
            update_graph([(r[1], r[2]) for r in refreshed], "Quotation Quantities", chart_type="bar")
        except Exception as exc:
            conn.rollback()
            messagebox.showerror("Error", str(exc))

    # === Action Buttons ===
    btn_frame = Frame(data_window, bg="#96C3EB")
    btn_frame.pack(pady=20)

    Button(btn_frame, text="Confirm", font=('times new roman', 15, 'bold'), bg="green", fg="white", width=12, command=lambda: update_selected_rows("Confirmed")
          ).pack(side=LEFT, padx=40)

    Button(btn_frame, text="Cancel", font=('times new roman', 15, 'bold'),bg="red", fg="white", width=12,command=lambda: update_selected_rows("Cancelled")
          ).pack(side=LEFT, padx=40)

    Button(btn_frame, text="Close", font=('times new roman', 15, 'bold'), command=data_window.destroy, width=12
          ).pack(side=LEFT, padx=40)

def fetch_customer_quotations():
    cursor.execute("SELECT CustomerName, ProductName, Quantity, Price, QuotationDate, OrderStatus FROM Quotations")
    return cursor.fetchall()

def open_quotation_window(_=None):
    rows = fetch_customer_quotations()
    show_data_in_product_window(
        "Customer Quotations",
        rows,
        ["CustomerName", "ProductName", "Quantity", "Price", "QuotationDate", "OrderStatus"],
    )
    update_graph([(r[1], r[2]) for r in rows], "Quotation Quantities", chart_type="bar")


product_frame=Frame(root, bg='#2C3E50', bd=3, relief=RIDGE)
product_frame.place(x=520, y=310, height=170, width=230)
total_product_image=PhotoImage(file='delivery-box.png')
total_product_image_label=Label(product_frame, image=total_product_image, bg='#2C3E50')
total_product_image_label.pack()
total_product_label=Label(product_frame, text='Quotation', font=('times new roman', 15, 'bold') ,bg='#2C3E50', fg='white')
total_product_label.pack()
total_product_count_label=Label(product_frame, text='0', bg='#2C3E50',fg='white', font=('times new roman', 20, 'bold'))
total_product_count_label.pack()
make_frame_clickable(product_frame, "Quotation",  open_quotation_window)

def fetch_order_status():
    cursor.execute("Select OrderID, CustomerID, ProductID, Quantity, TotalAmount, OrderDate, Status, PaymentMethod, ShippingAddress FROM Orders")
    return cursor.fetchall()

def open_order_window(title):
    order_data = fetch_order_status() 
    show_data_in_window(title, order_data, ["OrderID", "CustomerID", "ProductID", "Quantity", "TotalAmount", "OrderDate", "Status", "PaymentMethod", "ShippingAddress"])
    update_graph([(row[0], row[3]) for row in order_data], "Order Quantities", chart_type="bar")


order_frame=Frame(root, bg='#2C3E50', bd=3, relief=RIDGE)
order_frame.place(x=375, y=500, height=170, width=230)
total_order_image=PhotoImage(file='check-out.png')
total_order_image_label=Label(order_frame, image=total_order_image, bg='#2C3E50')
total_order_image_label.pack()
total_order_label=Label(order_frame, text="Logistics & Dispatch", font=('times new roman', 15, 'bold') ,bg='#2C3E50', fg='white')
total_order_label.pack()
total_order_count_label=Label(order_frame, text='0', bg='#2C3E50',fg='white', font=('times new roman', 20, 'bold'))
total_order_count_label.pack()
make_frame_clickable(order_frame, "Logistics & Dispatch", open_order_window)

# Alfiya Code
def order_status1(root):
    # Client list with "All"
    client_names = ["All", "John Doe", "Jane Smith", "XYZ Corp", "XWT Corp", "XQR Corp", "XGF Corp", "XIU Corp", "XTU Corp", "XOP Corp", "XRT Corp", "PUT Corp"]
    
    # Dummy Data: (Bill No, Client ID, Client Name, Customer Type, Status, Amount, Time)
    dummy_data = [
        (101, "CL001", "John Doe", "Retail", "Fabrication", "$1500", "08/02/25 10:30"),
        (102, "CL002", "Jane Smith", "Wholesale", "Testing", "$3200", "08/02/25 11:00"),
        (103, "CL003", "XYZ Corp", "Corporate", "Delivered", "$7800", "08/02/25 12:16"),
        (104, "CL004", "XWT Corp", "Corporate", "Delivered", "$7800", "08/02/25 12:17"),
        (105, "CL005", "XQR Corp", "Corporate", "Delivered", "$7800", "08/02/25 12:18"),
        (106, "CL006", "XGF Corp", "Corporate", "Delivered", "$7800", "08/02/25 01:00"),
        (107, "CL007", "XIU Corp", "Corporate", "Delivered", "$7800", "10/02/25 12:15"),
        (108, "CL008", "XTU Corp", "Corporate", "Delivered", "$7800", "11/02/25 12:15"),
        (109, "CL009", "XOP Corp", "Corporate", "Delivered", "$7890", "12/02/25 12:15"),
        (110, "CL010", "XRT Corp", "Corporate", "Delivered", "$7876", "13/02/25 12:15"),
        (111, "CL011", "PUT Corp", "Corporate", "Delivered", "$7812", "14/02/25 12:15"),
    ]
    
    # Flow stages in order
    flow_stages = ["Order", "Fabrication", "Powder Coating", "Assembly", "Testing", "Ready for Dispatch", "Shipped", "Delivered"]
    stage_index_map = {stage: idx for idx, stage in enumerate(flow_stages)}
    
    # Create main window
    root1 = tk.Tk()
    root1.title("Order Status Tracker")
    root1.geometry("1200x750")
    root1.configure(bg='#f0f4f7')
    
    
    
    tk.Label(header_frame, text="Client Name:", bg='#d1e0e0', font=('Arial', 13, 'bold')).pack(side='left', padx=(10, 2))
    client_entry = ttk.Combobox(header_frame, width=20)
    client_entry['values'] = client_names
    client_entry.set("All")
    client_entry.pack(side='left')
    
    from_date = DateEntry(header_frame, width=12, date_pattern='dd/mm/yy')
    to_date = DateEntry(header_frame, width=12, date_pattern='dd/mm/yy')
    
    # Treeview Frame
    table_frame = tk.Frame(root1)
    table_frame.pack(fill='both', expand=True, pady=10)
    
    columns = ("Select", "Bill No", "Client ID", "Client Name", "Customer Type", "Status", "Amount", "Time")
    tree = ttk.Treeview(table_frame, columns=columns, show='headings')
    
    for col in columns:
        tree.heading(col, text=col, anchor='center')
        tree.column(col, anchor='center', width=120)
    
    def load_data(filtered_data):
        for item in tree.get_children():
            tree.delete(item)
        for row in filtered_data:
            values = ("",) + row
            tree.insert("", tk.END, values=values)
    
    load_data(dummy_data)
    
    def filter_by_client():
        name = client_entry.get()
        if not name:
            messagebox.showwarning("Input Required", "Please select a client name.")
            return
        if name == "All":
            load_data(dummy_data)
        else:
            filtered = [row for row in dummy_data if row[2] == name]
            load_data(filtered)
    
    def filter_by_date():
        from_dt = datetime.strptime(from_date.get(), "%d/%m/%y")
        to_dt = datetime.strptime(to_date.get(), "%d/%m/%y")
        selected_client = client_entry.get()
    
        filtered = []
    
        for row in dummy_data:
            order_time = datetime.strptime(row[6], "%d/%m/%y %H:%M")
            status = row[4]
    
            if status == "Delivered" and from_dt <= order_time <= to_dt:
                if selected_client == "All" or row[2] == selected_client:
                    filtered.append(row)
    
        if not filtered:
            messagebox.showinfo("No Results", "No delivered orders found for the selected filters.")
    
        load_data(filtered)
    
    def view_all():
        load_data(dummy_data)
    
    def view_delivered():
        delivered = [row for row in dummy_data if row[4] == "Delivered"]
        load_data(delivered)
    
    def export_data():
        file_path = tk.filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
        if not file_path:
            return
        try:
            with open(file_path, 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(columns[1:])
                for child in tree.get_children():
                    row = tree.item(child)['values'][1:]
                    writer.writerow(row)
            messagebox.showinfo("Success", "Data exported successfully!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to export data.\n{e}")
    
    tk.Button(header_frame, text="Search", bg='#007acc', fg='white', command=filter_by_client).pack(side='left', padx=10)
    tk.Label(header_frame, text="From:", bg='#d1e0e0', font=('Arial', 13, 'bold')).pack(side='left')
    from_date.pack(side='left', padx=5)
    tk.Label(header_frame, text="To:", bg='#d1e0e0', font=('Arial', 13, 'bold')).pack(side='left')
    to_date.pack(side='left', padx=5)
    tk.Button(header_frame, text="Filter", bg='#007acc', fg='white', command=filter_by_date).pack(side='left', padx=10)
    tk.Button(header_frame, text="Show Details", bg='green', fg='white').pack(side='left', padx=10)
    
    tree.pack(fill='both', expand=True)
    
    # Status Buttons Frame
    status_frame = tk.LabelFrame(root1, text="Update Status", bg='#f0f4f7', font=('Arial', 13, 'bold'))
    status_frame.pack(fill='x', padx=10, pady=5)
    
    # Flow tracker frame (bottom)
    flow_frame = tk.Frame(root1, bg='#ffffff', pady=10)
    flow_frame.pack(fill='x', padx=10, pady=10)
    
    flow_stage_labels = {}
    
    def create_flow_tracker():
        for widget in flow_frame.winfo_children():
            widget.destroy()
        for stage in flow_stages:
            lbl = tk.Label(flow_frame, text=stage, bg='lightgray', font=('Arial', 13, 'bold'), width=16, height=2, relief='ridge', bd=2)
            lbl.pack(side='left', padx=(5, 0))
            flow_stage_labels[stage] = lbl
            if stage != flow_stages[-1]:
                arrow = tk.Label(flow_frame, text='➔', font=('Arial', 13, 'bold'), bg='#ffffff')
                arrow.pack(side='left', padx=(0, 5))
    
    create_flow_tracker()
    
    # To track which stages have been clicked per order (key: Treeview item ID, value: set of stages clicked)
    clicked_stages = {}
    
    def update_flow_tracker(current_stage):
        # Highlight stages up to current_stage green
        current_index = stage_index_map.get(current_stage, -1)
        for idx, stage in enumerate(flow_stages):
            if idx < current_index:
                flow_stage_labels[stage].config(bg='green', fg='white')  # Completed stages green
            elif idx == current_index:
                flow_stage_labels[stage].config(bg='skyblue', fg='white')  # Current stage blue
            else:
                flow_stage_labels[stage].config(bg='lightgray', fg='black')  # Pending stages gray
    
    def show_stage_details(stage, order_data):
        # Show all stages from "Order" up to given stage for this order/client
        current_idx = stage_index_map.get(stage, -1)
        stages_up_to = flow_stages[:current_idx + 1]
        details = f"Details for client: {order_data[2]} (Bill No: {order_data[0]})\n\n"
        details += "Completed Stages:\n"
        for s in stages_up_to:
            details += f"- {s}\n"
        messagebox.showinfo(f"Stage '{stage}' Details", details)
    
    def on_stage_button_click(stage):
        selected_items = tree.selection()
        if not selected_items:
            messagebox.showwarning("Warning", "Please select an order from the table.")
            return
        item = selected_items[0]
        order = tree.item(item)['values'][1:]  # exclude "Select" column
        current_status = order[4]
    
        # Initialize clicked stages for this item if not exists
        if item not in clicked_stages:
            clicked_stages[item] = set()
    
        # If this stage was already clicked for this order, ignore click
        if stage in clicked_stages[item]:
            messagebox.showinfo("Info", f"Stage '{stage}' was already processed for this order.")
            return
    
        # If the clicked stage index is less than or equal to current order status index, allow updating and showing details
        order_status_index = stage_index_map.get(current_status, -1)
        clicked_stage_index = stage_index_map.get(stage, -1)
    
        if clicked_stage_index <= order_status_index:
            # Show details up to clicked stage
            show_stage_details(stage, order)
            # Mark stage as clicked (processed)
            clicked_stages[item].add(stage)
            # Update Treeview status to this stage if stage is ahead of current
            if clicked_stage_index > order_status_index:
                new_values = list(order)
                new_values[4] = stage  # update status
                tree.item(item, values=("", *new_values))
            # Update flow tracker for this stage
            update_flow_tracker(stage)
        else:
            messagebox.showwarning("Invalid", f"You cannot process stage '{stage}' before completing previous stages.")
    
    # Create stage buttons
    for stage in flow_stages:
        if stage == "Order":  # skip order button for status updates
            continue
        btn = tk.Button(status_frame, text=stage, bg='#007acc', fg='white', width=15, font=('Arial', 13, 'bold'),
                        command=lambda s=stage: on_stage_button_click(s))
        btn.pack(side='left', padx=5, pady=5)
    
    # Bottom buttons frame
    bottom_frame = tk.Frame(root1, bg='#f0f4f7')
    bottom_frame.pack(fill='x', pady=5)
    
    
    tk.Button(bottom_frame, text="Export Data", bg='purple', fg='white', width=15, font=('Arial', 13, 'bold'), command=export_data).pack(side='right', padx=10)
    tk.Button(bottom_frame, text="View Delivered", bg='green', fg='white', width=15, font=('Arial', 13, 'bold'), command=view_delivered).pack(side='right', padx=10)
    tk.Button(bottom_frame, text="View All", bg='blue', fg='white', width=15, font=('Arial', 13, 'bold'), command=view_all).pack(side='right', padx=10)
    
root.mainloop()